In [1]:
import os, sys
curr_dir = os.getcwd()
parent_dir = os.path.dirname(curr_dir)
sys.path.insert(0, parent_dir)

('/home/am/Documents/Software Development/10_Academy Training/week6/credit-scoring/notebooks',
 '/home/am/Documents/Software Development/10_Academy Training/week6/credit-scoring')

In [ ]:
from scripts.exploratory_data_analysis import ExploratoryDataAnalysis

data_path = '../data/data.csv'

# Exploratory Data Analysis
eda = ExploratoryDataAnalysis(data_path)
eda.overview_of_data()
eda.summary_statistics()
eda.distribution_of_numerical_features()
eda.distribution_of_categorical_features()
eda.correlation_analysis()
eda.identify_missing_values()
eda.outlier_detection()

In [ ]:
# Feature Engineering
from scripts.feature_engineering import FeatureEngineering

chunk_size = 10000
chunks = pd.read_csv(data_path, chunksize=chunk_size)

for i, chunk in enumerate(chunks):
    print(f"Processing chunk {i+1}")
    feature_eng = FeatureEngineering(chunk)
    feature_eng.create_aggregate_features()
    feature_eng.extract_features()
    feature_eng.encode_categorical_variables(method="one_hot")
    feature_eng.handle_missing_values(strategy="mean")
    feature_eng.handle_outliers(method="iqr")
    feature_eng.normalize_or_standardize(method="standardize")
    chunk_output_path = f"../data/processed_chunk_{i+1}.csv"
    feature_eng.save_processed_data(chunk_output_path)
